In [79]:
# Clean Jupyter environment
%reset -f

In [80]:
name = 'trainingplans/*/Self/Marathon/*/*.yaml'

In [ ]:
from garminworkouts.models.workout import Workout
import argparse
from garminworkouts.models.settings import settings

args = argparse.Namespace(trainingplan=name)
workouts, *_ = settings(args)
mileage, duration, tss, ECOs, Rdist, Rdists, day_min, day_max = Workout.load_metrics(workouts=workouts)

In [ ]:
import matplotlib.pyplot as plt

weeks = list(range(len(mileage)))
plt.figure(figsize=(12, 12))

# Filter out weeks with zero values
non_zero_weeks = [week for week, mile in zip(weeks, mileage) if mile > 0]
non_zero_mileage = [mile for mile in mileage if mile > 0]
non_zero_ECOs = [eco for eco in ECOs if eco > 0]
non_zero_duration = [dur for dur in duration if dur.total_seconds() > 0]

plt.subplot(3, 1, 1)
plt.bar(non_zero_weeks[::-1], non_zero_mileage[::-1])
plt.ylabel('Mileage (km)')
plt.title('Mileage per Week')
plt.gca().invert_xaxis()

plt.subplot(3, 1, 2)
plt.bar(non_zero_weeks[::-1], [d.total_seconds() / 3600 for d in non_zero_duration[::-1]])
plt.ylabel('Duration (hours)')
plt.title('Duration per Week')
plt.gca().invert_xaxis()

plt.subplot(3, 1, 3)
plt.bar(non_zero_weeks[::-1], non_zero_ECOs[::-1])
plt.xlabel('Week')
plt.ylabel('ECOs')
plt.title('Training load per Week')
plt.gca().invert_xaxis()

plt.figure(figsize=(12, 6))
bottom = [0] * len(non_zero_weeks)
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange']
labels = ['R0', 'R1', 'R2', 'R3', 'R3+', 'R4', 'R5', 'R6']

for i in range(len(Rdist)):
    values = [week[i] / 3600 for week in Rdists if sum(week) > 0]
    plt.bar(non_zero_weeks[::-1], values[::-1], bottom=bottom[::-1], color=colors[i], label=labels[i])
    bottom = [sum(x) for x in zip(bottom, values)]

plt.xlabel('Week')
plt.ylabel('Distribution (hours)')
plt.title('Zone Distribution per Week')
plt.legend()
plt.gca().invert_xaxis()

plt.figure(figsize=(12, 6))
plt.bar(range(len(Rdist)), [r / 3600 for r in Rdist])
plt.xlabel('Zones')
plt.ylabel('Distribution (hours)')
plt.title('Zone Distribution Plot')
plt.xticks(range(len(Rdist)), labels)
plt.show()